In [4]:
# Load modules we'll need.

from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import spectrogram
%matplotlib osx

In [2]:
# Load the data.

data = loadmat('EEG-1.mat')    # Load the EEG data
EEG = data['EEG'].reshape(-1)  # Extract the EEG variable
t = data['t'][0]               # ... and the t variablea

In [5]:
# Look at it.

plt.plot(t,EEG)
plt.xlabel('Time [s]')
plt.ylabel('EEG');

In [6]:
#Q. What is the sampling interval (dt)?

dt = t[2]-t[1]
print(dt)

0.001


In [7]:
#Q. What is the number of points in the data (N)?

N = np.size(EEG)
print(N)

2000


In [8]:
#Q. What is the total time of the observation (T)?

T = N*dt
print(T)

2.0


In [9]:
# Compute the spectrum "by hand".

# First, get the frequency values (here f >= 0)
fNQ = (1/dt)/2
fj = np.linspace(0,fNQ,int(N/2+1))

# Then, compute the Fourier transform "by hand".

x = EEG
X = np.ndarray(np.size(fj), complex);
for j in range( np.size(fj) ):
    X[j] = np.sum(x * np.exp(-2*np.pi*1j*fj[j]*t))

# And the spectrum,

Sxx = 2 * dt ** 2 / T * (X * np.conj(X))

# Plot it,

plot(fj, Sxx.real)
plt.xlim([0, 100])                          # Select frequency range
plt.xlabel('Frequency [Hz]')                # Label the axes
plt.ylabel('Power [$\mu V^2$/Hz]');

In [10]:
# Compute the power spectrum using the FFT function.

x  = EEG
xf = np.fft.fft(x)                        # Compute Fourier transform of x
Sxx = 2 * dt ** 2 / T * (xf * xf.conj())  # Compute spectrum
Sxx = Sxx[0:int(N / 2)].real              # Ignore negative frequencies

In [11]:
# Define the frequency axis

df = 1/T                   # Determine frequency resolution
fNQ = 1/dt / 2            # Determine Nyquist frequency
faxis = np.arange(0,fNQ,df)  # Construct frequency axis

In [12]:
# Plot the spectrum versus frequency.
# Q. What do you see?

#plt.figure()
plt.plot(faxis, Sxx)
plt.xlim([0, 100])                          # Select frequency range
plt.xlabel('Frequency [Hz]')                # Label the axes
plt.ylabel('Power [$\mu V^2$/Hz]');

In [13]:
# Plot the spectrum versus frequency on a decibel scale.
# Q. Now what do you see?

plt.figure()
plt.plot(faxis, 10 * np.log10(Sxx / max(Sxx)))   # Plot the spectrum in decibels.
plt.xlim([0, 100])                           # Select the frequency range.
plt.ylim([-60, 0])                           # Select the decibel range.
plt.xlabel('Frequency [Hz]')                     # Label the axes.
plt.ylabel('Power [dB]');

In [17]:
# Plot the spectrum versus frequency on a logarithmic frequency axis.
# Q. And now what do you see?

plt.figure()
plt.semilogx(faxis, 10 * np.log10(Sxx / max(Sxx)))   # Log-log scale
plt.xlim([df, 100])                                  # Select frequency range
plt.ylim([-60, 0])                                   # ... and the decibel range.
plt.xlabel('Frequency [Hz]')                         # Label the axes.
plt.ylabel('Power [dB]');

In [16]:
# Plot the spectrogram.

Fs = 1 / dt               # Define the sampling frequency,
interval = int(Fs)        # ... the interval size,
overlap = int(Fs * 0.95)  # ... and the overlap intervals

                          # Compute the spectrogram
f0, t0, Sxx0 = spectrogram(
    EEG,                  # Provide the signal,
    fs=Fs,                # ... the sampling frequency,
    nperseg=interval,     # ... the length of a segment,
    noverlap=overlap)     # ... the number of samples to overlap,
plt.pcolormesh(t0, f0, 10 * np.log10(Sxx0),
               cmap='jet')# Plot the result
plt.colorbar()            # ... with a color bar,
plt.ylim([0, 70])             # ... set the frequency range,
plt.xlabel('Time [s]')       # ... and label the axes
plt.ylabel('Frequency [Hz]');

In [18]:
# Apply the Hanning taper and look at the data.

x_tapered  = np.hanning(N) * x              # Apply the Hanning taper to the data.
plt.figure()
plt.plot(t,x)
plt.plot(t,x_tapered);

In [19]:
# Apply the Hanning taper and look at the spectrum.

xf_tapered = np.fft.fft(x_tapered)               # Compute Fourier transform of x.
Sxx_tapered = 2 * dt ** 2 / T * (xf_tapered * np.conj(xf_tapered))  # Compute the spectrum,
Sxx_tapered = np.real(Sxx_tapered[:int(N / 2)])                # ... and ignore negative frequencies.

plt.figure()
plt.semilogx(faxis,10*np.log10(Sxx))         # Plot spectrum of untapered signal.  
plt.semilogx(faxis,10*np.log10(Sxx_tapered)) # Plot spectrum vs tapered signal.
plt.xlim([faxis[1], 100])                    # Select frequency range,
plt.ylim([-70, 20])                          # ... and the power range.
plt.xlabel('Frequency [Hz]')                 # Label the axes
plt.ylabel('Power [$\mu V^2$/Hz]')

Text(0,0.5,'Power [$\\mu V^2$/Hz]')